2023-10-18 Due to the fact that I am prototyping a lot of code and changing approaches (multiindex column default to long defaul primarily), literate progrmaming style is failing, re. version one of this notebook is somewhat unsalvagable - a testament to why rushing always fails in the end. At some point I'll have to go through and fix it. in the meantime, copy code over and push ON.

In [ ]:
# set up environment

%reload_ext autoreload
%autoreload 2

from dtwalign import dtw
import pandas as pd
from wine_analysis_hplc_uv import definitions
import seaborn as sns
import seaborn.objects as so
from wine_analysis_hplc_uv.notebooks.dtw_methods import DTWNotebookMethods
from wine_analysis_hplc_uv.signal_processing.mindex_signal_processing import (
    SignalProcessor,
)
from pybaselines import Baseline
import seaborn as sns
import matplotlib.pyplot as plt
from wine_analysis_hplc_uv.notebooks import eda_by_category_methods
plotter = eda_by_category_methods.Plotting()

scipro = SignalProcessor()

data = pd.read_parquet(definitions.RAW_PARQ_PATH)
data.head()

As we can see, no effect. The smoothing necessary to remove those detected peaks will result in unsatisfactory loss of signal information. Ergo better to use constraints in the peak detection algo. Also, remaining in simple long form with no multiindex massively reduces reshaping overhead and makes UDF functions much simpler to define..

Now lets add kwargs for peak finder.. added.

In [ ]:
# time std and drop vars level

pdata = (
    data.dropna()
    .pipe(scipro.standardize_time)
    .pipe(lambda df: df.set_axis(axis=0, labels=df.index.total_seconds() / 60))
    .droplevel("vars", axis=1)
)
pdata.head()

In [ ]:
# smooth
from scipy import signal

qdata = (
    pdata.melt(ignore_index=False, value_name="signal").reset_index()
    # smooth signal with window length 5 and polynomial order 2
    .assign(
        smooth_signal=lambda df: df.groupby("samplecode")["signal"].transform(
            func=signal.savgol_filter, **dict(window_length=5, polyorder=2)
        )
    )
)

qdata.head()

In [ ]:
# add bline


def bline(x, y, i):
    return pd.Series(Baseline(x, assume_sorted=True).asls(y)[0], index=i)


rdata = qdata.assign(
    bline=qdata.groupby("samplecode")["smooth_signal"].transform(
        lambda x: Baseline(x.index).asls(x)[0]
    )
).eval("bcorr=smooth_signal-bline")
rdata.head()

In [ ]:
# plot baseline correction

(
    rdata.melt(
        id_vars=["samplecode", "wine", "mins"], var_name="siglabel", value_name="sig"
    ).pipe(
        lambda df: so.Plot(data=df, x="mins", y="sig", color="siglabel")
        .facet(col="samplecode", wrap=2)
        .add(mark=so.Line())
    )
)

In [ ]:
# detect peaks prior to alignment

sdata = rdata.assign(
    bcorr_peaks=lambda df: df.groupby("samplecode")["bcorr"].transform(lambda x: x.iloc[signal.find_peaks(x)[0]])
)
sdata.head()

In [ ]:
# plot to display peaks
(
    sdata.melt(
        id_vars=["samplecode", "wine", "mins"], var_name="siglabel", value_name="sig"
    )
    .loc[lambda df: (df.mins < 5) & (df.mins > 3)]
    .pivot_table(index=["samplecode", "wine", "mins"], columns="siglabel", values="sig")
    .reset_index()
    .pipe(
        lambda df: so.Plot(data=df, x="mins", color="samplecode")
        .add(so.Line(), y="bcorr")
        .add(so.Dot(), y="bcorr_peaks")
    )
)

For testing purposes, we're only interested in the top 10 peaks

In [ ]:
# get top 20 peaks of the unaligned set

tdata = sdata.assign(
    bcorr_top_20=lambda df: df.groupby("samplecode", group_keys=False)[
        "bcorr_peaks"
    ].nlargest(20)
)

# plot the peaks on top of the curves
display(
    tdata.loc[lambda df: df.mins < 21].pipe(
        lambda df: (
            so.Plot(df, x="mins", color="samplecode")
            .add(so.Line(), y="bcorr")
            .add(so.Dot(), y="bcorr_top_20")
            .plot()
        )
    )
)

# peak table
(
    tdata.pipe(
        lambda df: (
            df
            if display(
                df.loc[:, ["samplecode", "wine", "mins", "bcorr_top_20"]]
                .dropna()
                .assign(n_peak=lambda df: df.groupby("samplecode").cumcount())
                .pivot(
                    columns=["samplecode", "wine"],
                    index=["n_peak"],
                    values=["mins", "bcorr_top_20"],
                )
                .reorder_levels([1, 2, 0], axis=1)
                .sort_index(axis=1)
            )
            else df
        )
    )
)

Now align, get the top 10 peaks after alignment and compare them.

Notes: Use transform with a custom function if you are expecting a result with the same shape as the input i.e. a column of the dataframe. Apply rarely gets this right.

In [ ]:
# define a class to handle DTW methods

class ApplyDTW():
    
    def align(self, data, primary_key, siglabel, aligned_label: str, kwargs:dict=dict()):
        """
        For a passed long format df find the reference signal and align, assigning back
        to the passed frame
        
        TODO: add internal validation
        """
        
        ref_idx = self.find_ref(data, primary_key, siglabel)
        
        
        ref_signal = data.loc[lambda df: df.samplecode==ref_idx][siglabel]
        
        adata = (
            data
            .assign(**{aligned_label:data
                       .groupby(primary_key)[siglabel]
                       .transform(self.applydtw, ref_signal)
                    }
                       )
                           )
        
        return adata
        
    
    def applydtw(self, query, ref, kwargs: dict = dict()):
        
        """
        apply dtw
        """
        # subset signal by warping path
        # reset index to enable reindexing after Apply functions (i.e. transform), 
        # otherwise get duplicate index error
        
        aligned_query = (query.iloc[dtw(x=query, y=ref, **kwargs).get_warping_path()]
        .reset_index(drop=True).values)
        
        aligned_s = pd.Series(aligned_query, index=query.index)
        
        return aligned_query
    
    def find_ref(self, data, primary_key:str, siglabel:str)->str:
        """
        pass long form df and return the idx string for the reference sample
        """
        
        tidy_data = data.pivot_table(index='mins',columns=primary_key, values=siglabel)

        
        ref_idx = (
            tidy_data
            .corr()
            .mean()
            .loc[lambda df: df == df.max()]
            .index[0]
        )
        return ref_idx
        
        


In [ ]:
# ref = ApplyDTW().find_ref(tdata, 'samplecode','bcorr')
align = ApplyDTW().align(tdata, primary_key='samplecode',siglabel='bcorr', aligned_label='aligned')

In [ ]:
display(align)

In [ ]:
# align the dataset

udata = dtwalignment(grouper=['samplecode','wine'], data=tdata, signal_label='bcorr', dtw_kwargs=dict(window_type='sakoechiba',window_size=500), ref_label=reference, aligned_signal_name='aligned_sig_1')
udata.head()

In [ ]:
# plot aligned dataset

(
    udata
    .loc[lambda df: df.mins<20]
    .pipe(
        lambda df: so.Plot(data=df, x="mins",color='samplecode')
        .layout(size=(5,3))
        .add(
            so.Line(), y="aligned_sig_1"
        )
        
    )
    )

In [ ]:
# plot aligned against unaligned set
(
    udata
    .loc[lambda df: df.mins<20]
    .melt(id_vars=['mins','samplecode','wine'],var_name='siglabel',value_vars=['bcorr','aligned_sig_1'],value_name='sig_val')
    .pipe(
        lambda df: so.Plot(data=df, x="mins",color='siglabel')
        .layout(size=(15,10))
        .add(
            so.Line(), y="sig_val"
        )
        .facet(col='samplecode',wrap=2)
    )
        
)

In [ ]:
# detect peaks in aligned set

vdata = (udata
 .assign(
    peaks_aligned=lambda df: df.groupby("samplecode")["aligned_sig_1"].transform(lambda x: x.iloc[signal.find_peaks(x)[0]])
)
)
vdata.head()

In [ ]:
# add column of top 20 peaks in aligned peak set

wdata = vdata.assign(
    aligned_top_20=lambda df: df.groupby("samplecode", group_keys=False)[
        "peaks_aligned"
    ].nlargest(20)
)
wdata.info()

In [ ]:
# construct and display peak table

aligned_peak_table = (
    wdata.pipe(
        lambda df: 
                df
                .loc[:,['mins','samplecode','wine','aligned_top_20']]
                .dropna()
                .assign(n_peak=lambda df: df.groupby("samplecode").cumcount())
                .pipe(lambda df: df if display(df) else df) # display df
                .pivot_table(
                    columns=["samplecode", "wine"],
                    index=["n_peak"],
                    values=["mins", "aligned_top_20"],
                )
                .reorder_levels([1, 2, 0], axis=1)
                .sort_index(axis=1)
        )
    )
aligned_peak_table

In [ ]:
# plot overlay signals and top 20 peaks

(
    wdata
    .loc[lambda df: df.mins<20]
    .pipe(lambda df:
        so.Plot(df, x='mins', color='samplecode')
        .add(so.Line(),y='aligned_sig_1')
        .add(so.Dot(), y='aligned_top_20')
        .facet(col='samplecode',wrap=2)
        .layout(size=(15,10))
        )
    )

In [ ]:
# plot overlay aligned set, unaligned peaks, aligned peaks as dots

plot_peak_data = wdata.melt(id_vars=['mins','samplecode','wine'],value_vars=['aligned_top_20','bcorr_top_20'], var_name='peak_label',value_name='sig').dropna()

(
    wdata
    .loc[lambda df: df.mins<20]
    .pipe(lambda df:
        so.Plot(df, x='mins')
        .add(so.Line(), y='aligned_sig_1',color='samplecode')
        .add(so.Dot(),data=plot_peak_data.loc[lambda df: df.mins<20], alpha='peak_label', color='samplecode',x='mins',y='sig')
        # .facet(col='samplecode',wrap=2)
        .layout(size=(15,10))
        )
    )

As we can see in the above graphic, The majority of the peaks have lost amplitude, and torbreck-struie has straight up lost a peak at 18 mins. We need a better DTW algorithm.

Need to develop a more reliable alignment method. But we also should look at whether torbreck-struie itself is the issue. First what happens if we exclude it during warping?